In [1]:
import CodaPCA
import CodaCl 
import PCACl
import numpy as np
from runpca import read_csv
import os
import sklearn
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
#change module for newer sklearn versions
from sklearn.model_selection  import cross_val_score
from sklearn.model_selection  import KFold
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
import torch
import dill

In [2]:


test_dict = {}

test_dict[(1,2,3)] = 1
test_dict[(1,2,3)] = [1,3]



    


In [3]:
def enhanced_cross_val(features, targets, folds):
    assert len(features) == len(targets), "Mismatch in length of features and targets"
    kfold_scores = []
    for train, test in folds:
        Y_train = targets[train]
        X_train = features[train]
               
        Y_test = targets[test]
        X_test = features[test]
                
        model = LogisticRegression(multi_class='auto', solver='lbfgs')
        model.fit(X_train, Y_train)
        y_pred = model.predict(X_test)
        kfold_scores.append(sklearn.metrics.accuracy_score(Y_test,y_pred))
                
    return kfold_scores

#can automate this if we had assume a certain structure for the indices of features and targets, or an array per dataset 




In [4]:
def coda_val(features, targets, n_components, folds, nn_shape,  lr,lam, epochs):
    kfold_scores = []
    for train, test in folds:        
        Y_train = targets[train]
        X_train = features[train]
        
       
        Y_test = targets[test]
        X_test = features[test]
        
        
        model = PCACl.CoDA_Cl(13, n_components, 3, nn_shape[0], nn_shape[1])


        val_arr, train_arr = model.fit(X_train, Y_train, lam, lr, train_size = int(len(X_train)*(3/4)), epochs=epochs)
        
#         print ("Plot!")
        
#         plt.plot(val_arr, c="red")
#         plt.plot(train_arr, c="blue")
        
#         plt.ylim(bottom=min(train_arr))

#         plt.show()
        
        pred = model.predict(torch.FloatTensor(X_test))
        
        pred = pred.exp().detach()     
        _, index = torch.max(pred,1)  
        pred = pred.numpy()
        index = index.numpy()

        
        
        
        kfold_scores.append(sklearn.metrics.accuracy_score(Y_test,index))
                
    return kfold_scores


In [5]:
def PCA_Classification(data, co_feature_indices, target_index, 
                   other_feature_indices = [], alg=CodaPCA.Alg.CODAPCA, verbose=True):
       #can loop through/optimise this in another way?
    
    features = data[:,co_feature_indices]
    
    features_ = features
    
    targets = data[:,target_index]
    
    
    #set up as int list, and subtract to get index so NLL loss works
    targets = list(map(int,data[:,0]))
    targets = [target - 1 for target in targets]

    
    #normalise the compositional features. TODO anything extra to deal with non compositional features?
    #features = np.array([feat/sum(feat) for feat in features])

    #can be empty
    extra_features = data[:,other_feature_indices]
    
    #TODO double check this
    features = np.hstack([features, extra_features])
    

    n_components=2
    

    pca_coda = CodaPCA.NonParametricCodaPCA(n_components)
    pca_coda.fit(features)
    
    Y_coda = pca_coda.transform(features)

    pca_clr = CodaPCA.CLRPCA(n_components)
    pca_clr.fit(features)
    
    Y_clr = pca_clr.transform(features)
    
    
    
    sc = StandardScaler()
    features_ = sc.fit_transform(features)

    
    pca_reg = decomposition.PCA(n_components)
    pca_reg.fit(features_)
    
    Y_pca = pca_reg.transform(features_)

    splits = 4
    #split data 
    kf = KFold(splits)
        
    folds = [i for i in kf.split(features)] 

    coda_score = enhanced_cross_val(Y_coda, np.array(targets), folds)
    clr_score = enhanced_cross_val(Y_clr, np.array(targets), folds) 
    naive_score = enhanced_cross_val(Y_pca, np.array(targets), folds)

#     import itertools
#     nn_vals = [[[5,], [2,]]]#, [[10,],[5,]], [[10,5], [3,5]], [[50,], [20,]], [[50,20], [10,20]]]
#     lr_vals = [1e-3]#, 1e-3, 5e-4,1e-4 ]
#     lam_vals = [1e-12]#1e-12,5e-12,1e-11,5e-11,1e-10,5e-10,1e-5,1e-4,0.001,0.1,1,10,100]
#     epoch_vals = [5000]

#     param_list = list(itertools.product(*[nn_vals, lr_vals, lam_vals, epoch_vals]))
# #     param_file = open("paramresultswineclass.txt","w") 
# #     param_file.write("Results from hyperparameter grid search:\n\n") 

#     param_dict = {}
        
#     for params in param_list:
#         nn_shape = params[0]
#         lr = params[1]
#         lam = params[2]
#         n_epochs = params[3]

#         score = coda_val(features, np.array(targets), 2, folds, nn_shape, lr, lam, n_epochs )

#         param_dict[tuple([sum(nn_shape[0]) + sum(nn_shape[1]), lr, lam, n_epochs])] = np.mean(score)
#         param_file.write("nn {}, lr {}, lam {}, epochs {}, num_samples {}, num_features {} \nScore: {} \nMean_score: {} \n\n".format(nn_shape, lr, lam, n_epochs, len(features), len(co_feature_indices), score, np.mean(score)))

#         param_file.close()

    #codacl_score = coda_val(features, np.array(targets), n_components, folds)

    nn_shape = [[5,], [2,]]
    lr = 1e-3
    lam = 1e-3
    n_epochs = 10000

    codacl_score = coda_val(features, np.array(targets), 2, folds, nn_shape, lr, lam, n_epochs )
    
    if verbose:
        print("CoDA-PCA:")
        print(coda_score)
        print("CLR-PCA:")
        print(clr_score)
        print ("Naive Classification:")
        print (naive_score)
        print ("Coda Cl:")
        print (codacl_score)

    return coda_score,clr_score,naive_score,codacl_score


In [6]:
wine = pd.read_csv("wine.data", header=None)

data = wine.to_numpy()
np.random.shuffle(data)


In [7]:
# %matplotlib inline
# param_dict = PCA_Classification(np.array(data), [1,2,3,4,5,6,7,8,9,10,11,12,13], 0)

# with open('param_search.pik', 'wb') as f:
#     dill.dump(param_dict, f)


In [8]:
# file = open('param_search.pik','rb')

# c = dill.load(file)

In [9]:
PCA_Classification(np.array(data), [1,2,3,4,5,6,7,8,9,10,11,12,13], 0)



epoch 1000, loss 8.869213104248047
epoch 2000, loss 8.458991050720215
epoch 3000, loss 8.4513578414917
epoch 4000, loss 8.451574325561523
epoch 5000, loss 8.45173454284668
epoch 6000, loss 8.451714515686035
epoch 7000, loss 8.451815605163574
epoch 8000, loss 8.451837539672852
epoch 9000, loss 8.451715469360352
epoch 10000, loss 8.451711654663086
epoch 1000, loss 8.476215362548828
epoch 2000, loss 8.350381851196289
epoch 3000, loss 8.326128959655762
epoch 4000, loss 8.3137845993042
epoch 5000, loss 7.9741411209106445
epoch 6000, loss 7.959311008453369
epoch 7000, loss 7.953436851501465
epoch 8000, loss 7.95097541809082
epoch 9000, loss 7.949704170227051
epoch 10000, loss 7.948823928833008
epoch 1000, loss 8.4981689453125
epoch 2000, loss 1.1112278699874878
epoch 3000, loss 1.1050199270248413
epoch 4000, loss 1.1042144298553467
epoch 5000, loss 1.1040921211242676
epoch 6000, loss 1.1041102409362793
epoch 7000, loss 1.1044622659683228
epoch 8000, loss 1.1041207313537598
epoch 9000, loss 1

([0.7555555555555555,
  0.8666666666666667,
  0.7954545454545454,
  0.7727272727272727],
 [0.8888888888888888,
  0.9777777777777777,
  0.8409090909090909,
  0.9545454545454546],
 [0.9777777777777777,
  0.9333333333333333,
  0.9545454545454546,
  0.9772727272727273],
 [0.9777777777777777,
  0.9333333333333333,
  0.6363636363636364,
  0.8636363636363636])